In [ ]:
class GlobalClock(object):

    ##
    #
    ##
    def __init__(self, startTime, stationMap, tripGenerator,
                 imageDir=None, imageFreq=20, vehicles=[],
                 verbosity=1, durList={}):
        ''' IMPORTANT: initialized to start from 7am'''
        self.time = startTime
        self.heap = []

        self.statMap = stationMap
        self.tripGen = tripGenerator

        self.trips = []

        self.vehicles = vehicles

        self.imageDir = imageDir
        self.imageFreq = imageFreq
        self.imageCount = 0

        self.verbosity = verbosity

        self.durList = durList

    def printTripStatus(self, ):
        print self.time,
        print len(filter(lambda x: x.state is Trip.TRIPOK, self.trips)),
        print len(filter(lambda x: x.state is Trip.TRIPENDOK, self.trips)),
        print len(filter(lambda x: x.state is Trip.STARTFAILED, self.trips)),
        print len(filter(lambda x: x.state is Trip.ENDFAILED, self.trips)),
        print len(filter(lambda x: x.state is Trip.TRIPENDBAD, self.trips)),
        print len(self.trips)

    def getTripOk(self, ):
        return len(filter(lambda x: x.state is Trip.TRIPENDOK, self.trips))

    def getFailedStarts(self, ):
        return len(filter(lambda x: x.state is Trip.STARTFAILED, self.trips))

    def getCompletedTrips(self, ):
        return len(filter(lambda x: x.state in [Trip.TRIPENDOK, Trip.ENDFAILED], self.trips))

    def getFailedEnds(self, ):
        return len(filter(lambda x: x.state is Trip.TRIPENDBAD, self.trips))

    # Caution: If a biker visits multiple stations before finding a dock,
    #          this is only counting the first one visited.
    def getAlterEnd(self, ):
        return len(filter(lambda x: x.state is Trip.ENDFAILED, self.trips))

    def getOutageMins(self,):
        return sum(map(lambda x: sum(x.getOutageMins()), self.statMap.values()))

    def getTotalTrips(self, ):
        return len(self.trips)

    def getEmptyMins(self, ):
        return sum(map(lambda x: sum(x.getEmptyMins()), self.statMap.values()))

    def getFullMins(self, ):
        return sum(map(lambda x: sum(x.getFullMins()), self.statMap.values()))

    ''' Editted to return station with max failed starts and max failed ends'''

    def getFailedStartsStation(self, ):
        slist = map(lambda x: x.startLoc, filter(
            lambda x: x.state is Trip.STARTFAILED, self.trips))
        if len(slist) == 0:
            return -1
        else:
            return slist

    def getAlterEndsStation(self, ):
        slist = map(lambda x: x.failedEndSid, filter(
            lambda x: x.failedEndSid is not None, self.trips))
        if len(slist) == 0:
            return -1
        else:
            return slist

    # ''' Editted to return station with max empty mins and max full mins '''
    # def getMaxEmptyMinsStat(self, ):
    #     listE = map(lambda x: sum(x.getEmptyMins()), self.statMap.values())
    #     maxE = listE.index(max(listE))
    #     return Array(self.statMap)[maxE].0

    # def getMaxFullMinsStat(self, ):
    #     listF = map(lambda x: sum(x.getFullMins()), self.statMap.values())
    #     maxF = listE.index(max(listF))
    #     return Array(self.statMap)[maxF].0

    def getScatterPlotData(self, result={}, numsamples=1):

        for trip in filter(lambda x: x.state is Trip.TRIPENDOK, self.trips):
            oID = trip.startLoc
            dID = trip.endLoc
            if not result.has_key(oID):
                result[oID] = {}
            if not result[oID].has_key(dID):
                result[oID][dID] = 1/float(numsamples)
            else:
                result[oID][dID] += 1/float(numsamples)
        return result

    def getScatterPlotDataTimes(self, result={}, numsamples=1):

        for trip in filter(lambda x: x.state in
                           [Trip.TRIPENDOK, Trip.ENDFAILED], self.trips):
            startTime = trip.startTime
            endTime = trip.endTime

            if not result.has_key(startTime/10):
                result[startTime/10] = [1/float(numsamples), 0]
            else:
                result[startTime/10][0] += 1/float(numsamples)

            if not result.has_key(endTime/10):
                result[endTime/10] = [0, 1/float(numsamples)]
            else:
                result[endTime/10][1] += 1/float(numsamples)

        return result

    def tick_original(self, minTrips):
        if self.verbosity is 1:
            self.printTripStatus()

        # Get new trips to start
        badEndTrips = []

        # First we process what can happen at the current time
        toProc = []
        while len(self.heap) > 0 and self.heap[0] < self.time:
            # Now process these, assume they are all trips for now
            trip = heappop(self.heap)
            # mismatch statMap: may not have endLoc
            if trip.endLoc is not None and self.statMap.has_key(trip.endLoc):
                tin = self.statMap[trip.endLoc].tripIn(trip)
                if tin is not None:
                    badEndTrips.append(tin)

        # Then we make new end point trips happen
        for trip in badEndTrips:
            heappush(self.heap, trip)

        # Then we make new trips happen
        # numpy.random.seed(Config.seeds[0])
        tripList = minTrips
        for trip in tripList:
            tok = self.statMap[trip.startLoc].tripOut(trip)
            self.trips.append(trip)
            if tok:
                heappush(self.heap, trip)

        for stat in self.statMap.values():
            stat.tick(self.time)

        # Now we deal with rebalancing
        # for v in self.vehicles: v.tick(self.time, self.statMap)

        # Should we dump out an image
        if self.imageDir is not None:
            if self.time % self.imageFreq is 0:
                self.dumpImage()

        # for sid in [276, 251]:
        #     fileName = ("./outputsDO/lvlInDay/sid"+str(sid)+"lvlInDay"+str(rep)+".p")
        #     levels = cPickle.load(open(fileName))
        #     levels[self.time] = self.statMap[sid].level
        #     cPickle.dump(levels, open(fileName, 'wb'))

        # At the end of tick we increment the time
        self.time += 1

    def dumpImage(self):
        gmap = MapMaker.Map()
        for stat in self.statMap.values():
            if stat.cap > 0:
                pct = stat.level/float(stat.cap)
                gmap.addCircle(MapMaker.Circle(stat.ords, pct=pct))
        gmap.produceImage(self.imageDir+"/%d_state.png" % self.imageCount,
                          "%02d:%02d" % (self.time/60, self.time % 60))
        self.imageCount += 1

    def addTripToProcess(self, trip):
        heapq.heappush(self.heap, trip)


In [ ]:
class Station(object):

    def __init__(self, sid, name, cap, level, clock, ords=None):
        self.sid = sid
        self.name = name
        self.cap = cap
        self.level = level
        self.clock = clock
        self.ords = ords

        # Record state over time
        self.levelTime = []

        self.tripsOut = []
        self.tripsIn = []
        self.tripsFailedOut = []
        self.tripsFailedIn = []

    def tripIn(self, trip):
        if self.cap > self.level:
            trip.endTrip()
            self.level += 1
            return None
        else:
            return trip.failEnd()

    def tripOut(self, trip):
        if self.level == 0:
            trip.failStart()
            return False
        else:
            self.level -= 1
            return True

    def recordTime(self, time):
        self.levelTime.append(self.level)

    def tick(self, time): self.recordTime(time)

    def distanceTo(self, stat):
        return ((stat.ords[0] - self.ords[0])*(stat.ords[0] - self.ords[0]) +
                (stat.ords[1] - self.ords[1])*(stat.ords[1] - self.ords[1]))

    def getOutageMins(self, ):
        res = []
        for l in self.levelTime:
            if l == 0 or l == self.cap:
                res.append(1)
            else:
                res.append(0)
        return res

    def getEmptyMins(self, ):
        res = []
        for l in self.levelTime:
            if l == 0:
                res.append(1)
            else:
                res.append(0)
        return res

    def getFullMins(self, ):
        res = []
        for l in self.levelTime:
            if l == self.cap:
                res.append(1)
            else:
                res.append(0)
        return res

    def getSla(self):
        res = []
        for l in self.levelTime:
            if l == 0 or l == self.cap:
                res.append(1)
            else:
                res.append(0)
        return sum(res[6*60:22*60])


In [ ]:
class TripGenerator(object):
    # code

    def __init__(self, statMap, minbymin, durations, mult=1):
        self.statMap = statMap
        self.tid = 0
        self.minbymin = minbymin
        self.mult = mult
        self.durations = durations

    # runDay_alt2 method for generating trips

    def getAllTripsCombined(self, timeStart):
        # Initialize dictionary with keys = minutes in interval. Entry = list of trips for that minute
        trips = {}
        for i in range(30):
            trips[timeStart+i] = []

        for sid in self.statMap.keys():
            # First, check corner cases
            if not self.minbymin[timeStart].has_key(sid):
                continue
            if self.minbymin[timeStart][sid]["total"] == 0:
                continue
            if not self.statMap.has_key(sid):
                continue

            # Get trip counts for full 30 minute interval
            n = numpy.random.poisson(self.minbymin[timeStart][sid]["total"], size=30)
            totalTrips = sum(n)

            # Get list of all destinations
            total = self.minbymin[timeStart][sid]["total"]
            dests = self.minbymin[timeStart][sid]["dests"].keys()
            distr = map(lambda x: x/float(total),
                        self.minbymin[timeStart][sid]["dests"].values())
            ''' NJ: tolist() converts numpy array to list, so can use .index later'''
            res = numpy.random.multinomial(1, distr, size=totalTrips).tolist()

            ''' NJ: the creation of allDests and its two for loops can be avoided. See below '''
            # allDests = []
            # for r in range(totalTrips):
            #     for i in range(len(res[r])):
            #         if res[r][i] == 1: allDests.append(dests[i])

            # Assign trips to the dictionary entry for their respective minute
            count = 0
            for i in range(30):
                # Get trips for ith minute (time = timeStart + i)
                time = timeStart + i
                # j in number of trips at time
                for j in range(n[i]):
                    ''' NJ: .index(1) returns the index corresponding to the value 1. It is faster than a
                    for loop and can be used since there is only one 1'''
                    dest = dests[res[count].index(1)]
                    duration = self.getDuration(sid, dest, time)
                    newTrip = Trip(startTime=time, startLoc=sid,
                                   endTime=time + duration, endLoc=dest,
                                   duration=duration, clock=time, statMap=self.statMap,
                                   tgen=self)
                    # Add new trips to full trip dictionary under minute "time"
                    trips[time].append(newTrip)
                    count += 1

            # DEBUGGING
            if count != totalTrips:
                print "Error creating trip list"
        return trips

    def getAllTripsCombinedAlias(self, timeStart):
        # Initialize dictionary with keys = minutes in interval. Entry = list of trips for that minute
        trips = {}
        for i in range(30):
            trips[timeStart+i] = []

        for sid in self.statMap.keys():
            # First, check corner cases
            if not self.minbymin[timeStart].has_key(sid):
                continue
            if self.minbymin[timeStart][sid]["total"] == 0:
                continue
            if not self.statMap.has_key(sid):
                continue

            # Get trip counts for full 30 minute interval
            n = numpy.random.poisson(
                self.minbymin[timeStart][sid]["total"], size=30)
            totalTrips = sum(n)

            # Get list of all destinations
            total = self.minbymin[timeStart][sid]["total"]
            dests = self.minbymin[timeStart][sid]["dests"].keys()
            distr = map(lambda x: x/float(total),
                        self.minbymin[timeStart][sid]["dests"].values())
            # do the setup for alias once every 30 minutes
            J, q = alias_setup(distr)
            # draw all the destinations
            destIndex = alias_draw_n(J, q, totalTrips)
            ts = timeStart + numpy.random.randint(30, size=totalTrips)

            # Assign trips to the dictionary entry for their respective minute
            for i in range(totalTrips):
                time = int(ts[i])
                dest = dests[destIndex[i]]
                duration = self.getDuration(sid, dest, time)
                newTrip = Trip(startTime=time, startLoc=sid,
                               endTime=time + duration, endLoc=dest,
                               duration=duration, clock=time, statMap=self.statMap,
                               tgen=self)
                trips[time].append(newTrip)
        return trips

    def getAllTripsCombinedCond(self, timeStart):
        # Initialize dictionary with keys = minutes in interval. Entry = list of trips for that minute
        trips = {}
        for i in range(30):
            trips[timeStart+i] = []

        for sid in self.statMap.keys():
            # First, check corner cases
            if not self.minbymin[timeStart].has_key(sid):
                continue
            if self.minbymin[timeStart][sid]["total"] == 0:
                continue
            if not self.statMap.has_key(sid):
                continue

            # Get trip counts for full 30 minute interval
            totalTrips = numpy.random.poisson(
                self.minbymin[timeStart][sid]["total"]*30, size=1)

            # Get list of all destinations
            total = self.minbymin[timeStart][sid]["total"]
            dests = self.minbymin[timeStart][sid]["dests"].keys()
            distr = map(lambda x: x/float(total),
                        self.minbymin[timeStart][sid]["dests"].values())
            res = numpy.random.multinomial(1, distr, size=totalTrips).tolist()

            ts = timeStart + numpy.random.randint(30, size=totalTrips)
            # Assign trips to the dictionary entry for their respective minute
            for i in range(totalTrips):
                ''' Poisson Process property: conditioning on number of arrivals in [0,T],
                the time of arrivals is uniform in [0,T]. Here we use the discrete uniform.'''
                time = int(ts[i])
                dest = dests[res[i].index(1)]
                duration = self.getDuration(sid, dest, time)
                newTrip = Trip(startTime=time, startLoc=sid,
                               endTime=time + duration, endLoc=dest,
                               duration=duration, clock=time, statMap=self.statMap,
                               tgen=self)
                trips[time].append(newTrip)
        return trips

    ''' runDay_original methods for generating trips '''

    def generateTrips(self, timeValue):
        trips = []
        for s1 in self.statMap.keys():
            trips.extend(self.getTrips(s1, timeValue))
        return trips

    def getTrips(self, sid, time):
        if not self.minbymin[time-time % 30].has_key(sid):
            return []
        if self.minbymin[time-time % 30][sid]["total"] == 0:
            return []
        if not self.statMap.has_key(sid):
            return []

        numberTrips = numpy.random.poisson(self.minbymin[time-time % 30][sid]["total"] *
                                           self.mult)

        trips = []
        for t in range(numberTrips):
            dest = self.getWhereTripGoing(sid, time)
            duration = self.getDuration(sid, dest, time)
            newTrip = Trip(time, sid, time+duration, dest,
                           duration, time, statMap=self.statMap,
                           tgen=self)
            trips.append(newTrip)
        return trips

    def getWhereTripGoing(self, s1, time):
        total = self.minbymin[time-time % 30][s1]["total"]
        dests = self.minbymin[time-time % 30][s1]["dests"].keys()
        distr = map(lambda x: x/float(total),
                    self.minbymin[time-time % 30][s1]["dests"].values())
        res = numpy.random.multinomial(1, distr)
        for i in range(len(res)):
            if res[i] == 1:
                return dests[i]

    def getDuration(self, s1, s2, t, ind=None):
        # if not self.minbymin.has_key(t-t%30):
        #     # print "Warning: Time %d has no key" %t
        #     return 10
        # if not self.minbymin[t-t%30].has_key(s1):
        #     # print "Warning: sid=%d has no key at time %d" %(s1,t)
        #     return 10
        # if not self.minbymin[t-t%30][s1]["durations"].has_key(s2):
        #     # print "Warning: s1=%d has no duration key to s2=%d at time %d" %(s1,s2,t)
        #     return 10

        sigma = 0.2571  # sqrt of 0.0661117309208
        mult = self.durations[s1][s2]

        ''' If ind=None, this is called when generating the trip list, so return one duration. If ind = 1 or 2,
        meaning ind number of failedEnds has been triggered, return duration without affecting the
        random number cycle'''
        if ind is None:
            duration = int(round(mult*numpy.random.lognormal(0, sigma)))
        else:
            state = numpy.random.get_state()
            # numpy.random.RandomState() # Start a new stream
            duration = int(round(mult*numpy.random.lognormal(0, sigma)))
            numpy.random.set_state(state)  # Restore state
        return duration


In [ ]:
import random


def generate_trial_solution(x_k_minus_1, statE, statF, r, w):
    # randomly choose station from statE and statF that satisfies constraints
    sE = random.choice([s for s in statE if x_k_minus_1[s] + w <= r[s]])
    sF = random.choice([s for s in statF if x_k_minus_1[s] - w >= 0])

    # generate trial solution by adding w bikes to sE and removing w bikes from sF based on x_k_minus_1
    x_trial = x_k_minus_1.copy()
    x_trial[sE] += w
    x_trial[sF] -= w

    return x_trial


def simulate_and_evaluate(x, day_seed, m):
    # set random seed for simulation
    random.seed(day_seed)

    # simulate system and evaluate objective using sample average over m replications
    obj_sum = 0
    for i in range(m):
        # run simulation and obtain objective value
        obj = run_simulation(x)
        obj_sum += obj

    # calculate average objective value
    avg_obj = obj_sum / m

    return avg_obj


def optimize_x_for_morning_rush(x0, statE, statF, r, m, w):
    # initialize set
    k = 1

    # evaluate initial solution x0
    statE_k_minus_1, statF_k_minus_1 = simulate_and_evaluate(
        x0, random.randint(1, 1000), m)

    # repeat until stopping criterion is met
    while True:
        # generate trial solution
        x_trial = generate_trial_solution(x_k_minus_1, statE, statF, r, w)

        # simulate and evaluate trial solution
        statE_trial, statF_trial = simulate_and_evaluate(
            x_trial, random.randint(1, 1000), m)

        # check if trial solution improves objective
        if statE_trial < statE_k_minus_1 or statF_trial < statF_k_minus_1:
            # accept trial solution
            x_k = x_trial
            statE_k_minus_1, statF_k_minus_1 = statE_trial, statF_trial

            # increment k
            k += 1

            # record new statE and statF values
            statE.append(statE_k_minus_1)
            statF.append(statF_k_minus_1)
        else:
            # reject trial solution and try again
            continue

        # check stopping criterion
        if k >= max_iterations:
            break

    return x_k
